# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
#read and load in csv from Weather.py
#convert to dataframe
file = "../WeatherPy/WeatherPy-SampleCities.csv"
citysample_df = pd.read_csv(file)  
citysample_df = citysample_df.drop(columns='Unnamed: 0')
citysample_df

,long,lat,City,Country,Temp,Humidity,Cloudiness,Wind (mph)
0,128.379756,-64.324089,new norfolk,au,45.968,70,100,2.19
1,59.195332,-13.702434,cap malheureux,mu,64.004,82,92,0.89
2,-84.255361,-86.054528,ushuaia,ar,35.600,80,75,10.30
3,9.199472,29.322843,nalut,ly,76.442,53,82,6.86
4,53.140293,22.431181,abu dhabi,ae,89.600,75,1,2.60
...,...,...,...,...,...,...,...,...
442,179.187869,-50.011217,christchurch,nz,36.446,74,0,1.00
443,38.746392,52.649579,donskoye,ru,66.200,37,0,3.00
444,13.833985,-1.227258,lekoni,ga,77.036,78,100,2.92
445,-65.259670,38.322374,shelburne,ca,66.308,49,20,6.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
citysample_df

,long,lat,City,Country,Temp,Humidity,Cloudiness,Wind (mph)
0,128.379756,-64.324089,new norfolk,au,45.968,70,100,2.19
1,59.195332,-13.702434,cap malheureux,mu,64.004,82,92,0.89
2,-84.255361,-86.054528,ushuaia,ar,35.600,80,75,10.30
3,9.199472,29.322843,nalut,ly,76.442,53,82,6.86
4,53.140293,22.431181,abu dhabi,ae,89.600,75,1,2.60
...,...,...,...,...,...,...,...,...
442,179.187869,-50.011217,christchurch,nz,36.446,74,0,1.00
443,38.746392,52.649579,donskoye,ru,66.200,37,0,3.00
444,13.833985,-1.227258,lekoni,ga,77.036,78,100,2.92
445,-65.259670,38.322374,shelburne,ca,66.308,49,20,6.70


In [18]:
#humidity heatmap using google maps heat map info
coords = citysample_df[["lat","long"]]
humidity = citysample_df["Humidity"].astype(float)

figure_layout = { 
                'width' : '1000px',
                'height' : '600px',
                'border':'1px solid black',
                'padding':'1px',
                'margin':'0 auto 0 auto'
}


######################################################################
##  Created a map figure, added markers with nearest placenames for 
##  data set, and adjusted map for readability.
##  The map currently shows large color swatches to indicate regions of 
##  higher and lower humidity.
######################################################################

fig = gmaps.figure(layout=figure_layout)
info = citysample_df["City"]

##
## Add the two commented-out lines below to overlay markers on top of this map
##


#markers = gmaps.marker_layer(coords,info_box_content = "Randomly Generated Point, Nearest City :" + info + "")
#fig.add_layer(markers)


heatmap_layer = gmaps.heatmap_layer(coords, weights=humidity,max_intensity=1500)
heatmap_layer.point_radius = 30
heatmap_layer.dissipating=False
fig.add_layer(heatmap_layer)
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
## Get names of indexes for which column Age has value 30
#indexNames = dfObj[ dfObj['Age'] == 30 ].index
# Delete these row indexes from dataFrame
#dfObj.drop(indexNames , inplace=True)

In [20]:
#################################################
## Create a copy dataframe for data consistency.
#################################################

betterweather_df = citysample_df.copy()



#################################################################
## * A max temperature lower than 80 degrees but higher than 70.
#################################################################

index_min = betterweather_df[betterweather_df["Temp"] < 70].index
betterweather_df.drop(index_min, inplace=True)

index_max = betterweather_df[betterweather_df["Temp"] > 80].index
betterweather_df.drop(index_max, inplace=True)




#################################################################
##  * Wind speed less than 10 mph.
#################################################################

index_wind = betterweather_df[betterweather_df["Wind (mph)"] > 10].index
betterweather_df.drop(index_wind, inplace=True)



#################################################################
##  * Zero cloudiness.
#################################################################

index_cloud = betterweather_df[betterweather_df["Cloudiness"] > 0].index
betterweather_df.drop(index_cloud, inplace=True)




betterweather_df

,long,lat,City,Country,Temp,Humidity,Cloudiness,Wind (mph)
21,-137.847363,-32.735042,rikitea,pf,73.508,72,0,2.00
30,-116.906566,39.135700,fallon,us,75.992,20,0,1.06
82,-159.176738,-1.224683,faanui,pf,77.756,78,0,7.63
120,-23.271040,-24.747381,caravelas,br,78.674,70,0,3.58
145,23.613633,18.196267,kutum,sd,79.484,40,0,5.23
146,63.112246,30.444733,rudbar,af,70.160,22,0,3.34
190,44.576205,2.318491,wanlaweyn,so,77.666,77,0,7.51
192,86.047779,35.753045,korla,cn,73.994,63,0,0.89
406,114.282863,-5.740023,sumenep,id,78.728,68,0,3.58


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
hotel_df = betterweather_df.copy()
hotel_df["Hotel Name"] = ""
hotel_df.reset_index(inplace=True,drop=True)
hotel_df

,long,lat,City,Country,Temp,Humidity,Cloudiness,Wind (mph),Hotel Name
0,-137.847363,-32.735042,rikitea,pf,73.508,72,0,2.00,
1,-116.906566,39.135700,fallon,us,75.992,20,0,1.06,
2,-159.176738,-1.224683,faanui,pf,77.756,78,0,7.63,
3,-23.271040,-24.747381,caravelas,br,78.674,70,0,3.58,
4,23.613633,18.196267,kutum,sd,79.484,40,0,5.23,
5,63.112246,30.444733,rudbar,af,70.160,22,0,3.34,
6,44.576205,2.318491,wanlaweyn,so,77.666,77,0,7.51,
7,86.047779,35.753045,korla,cn,73.994,63,0,0.89,
8,114.282863,-5.740023,sumenep,id,78.728,68,0,3.58,


In [32]:
#-23.121533, -134.969740

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "39.952583,-75.16522",  # philadelphia coords,
    "rankby": "distance",
    "type": "restaurant",
    "key": g_key,
}

print(f"Retrieving Results for {params['location']}")
response = requests.get(base_url, params=params).json()
    
    # extract results
results = response['results']
    
if results == []:
    print("No results were found for those coordinates.")
        
else:
    print(f"Closest hotel is {results[0]['name']}.")
    #hotel_df.loc[index, 'Hotel Name'] = results[0]['name']

Retrieving Results for 39.952583,-75.16522
Closest hotel is Hassoumi Boureiam.


In [37]:
#-116.906566 39.135700
#-137.847363 -32.735042

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "39.135700,-116.906566",  # french polynesia coords,
    #"rankby": "distance",
    "type": "hotel",
    "key": g_key,
    "radius":50000
}

print(f"Retrieving Results for {params['location']}")
response = requests.get(base_url, params=params).json()
    
    # extract results
results = response['results']
    
if results == []:
    print("No results were found for those coordinates.")
        
else:
    print(f"Closest hotel is {results[0]['name']}.")
    #hotel_df.loc[index, 'Hotel Name'] = results[0]['name']

Retrieving Results for 39.135700,-116.906566
Closest hotel is Round Mountain.


In [41]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#params = {
#    "location": "39.952583,-75.16522",  # philadelphia coords,
#    #"rankby": "distance",
#    "type": "lodging",
#    "key": g_key,
#    "radius":50000
#}

for index, row in hotel_df.iterrows():

    
    gp_long = row['long']
    gp_lat = row['lat']
    #location = (gp_lat,gp_long)
    
    params = {
    #"location": "39.135700, -116.906566",
    "location":f"{gp_lat},{gp_long}",
    #"location": "39.135700,-116.906566",
        
    #"latitude":'39.135700',
    #"longitude":'-116.906566',
    #"location":location,  # philadelphia coords,
    #"rankby": "distance",
    "type": "lodging",
    "key": g_key,
    "radius":50000
    }

   

    # assemble url and make API request
    #print(f"Retrieving Results for { params['latitude']} and {params['longitude']}")
    print(f"Retrieving Results for {params['location']}")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    if results == []:
        print("No results were found for those coordinates.")
        hotel_df.loc[index,'Hotel Name'] = ""
        
    else:
        print(f"Closest restaurant is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']

Retrieving Results for -32.735041665008985,-137.84736312141652
No results were found for those coordinates.
Retrieving Results for 39.135699928629045,-116.90656647039988
Closest restaurant is Kingston Campground.
Retrieving Results for -1.2246830960241262,-159.1767381159511
No results were found for those coordinates.
Retrieving Results for -24.74738119985893,-23.27104015781536
No results were found for those coordinates.
Retrieving Results for 18.196266530547188,23.61363250295689
No results were found for those coordinates.
Retrieving Results for 30.44473272156224,63.11224574594798
No results were found for those coordinates.
Retrieving Results for 2.3184905486212988,44.57620494497137
Closest restaurant is Haji Qasim Hotel.
Retrieving Results for 35.75304502063753,86.04777892579003
No results were found for those coordinates.
Retrieving Results for -5.740023127893879,114.28286337697985
Closest restaurant is TPU masalima.


In [42]:
hotel_df

,long,lat,City,Country,Temp,Humidity,Cloudiness,Wind (mph),Hotel Name
0,-137.847363,-32.735042,rikitea,pf,73.508,72,0,2.00,
1,-116.906566,39.135700,fallon,us,75.992,20,0,1.06,Kingston Campground
2,-159.176738,-1.224683,faanui,pf,77.756,78,0,7.63,
3,-23.271040,-24.747381,caravelas,br,78.674,70,0,3.58,
4,23.613633,18.196267,kutum,sd,79.484,40,0,5.23,
5,63.112246,30.444733,rudbar,af,70.160,22,0,3.34,
6,44.576205,2.318491,wanlaweyn,so,77.666,77,0,7.51,Haji Qasim Hotel
7,86.047779,35.753045,korla,cn,73.994,63,0,0.89,
8,114.282863,-5.740023,sumenep,id,78.728,68,0,3.58,TPU masalima


In [43]:
hotelspots_df = hotel_df.copy()


index_hotels = hotelspots_df[hotelspots_df["Hotel Name"] == ""].index
hotelspots_df.drop(index_hotels, inplace=True)


hotelspots_df

,long,lat,City,Country,Temp,Humidity,Cloudiness,Wind (mph),Hotel Name
1,-116.906566,39.135700,fallon,us,75.992,20,0,1.06,Kingston Campground
6,44.576205,2.318491,wanlaweyn,so,77.666,77,0,7.51,Haji Qasim Hotel
8,114.282863,-5.740023,sumenep,id,78.728,68,0,3.58,TPU masalima


In [44]:
#Google Places API for each city's coordinates, search for hotel within 5000 meters, store the first hotel returned 
#into the dataframe under Hotel Name


In [45]:
hotel_df

,long,lat,City,Country,Temp,Humidity,Cloudiness,Wind (mph),Hotel Name
0,-137.847363,-32.735042,rikitea,pf,73.508,72,0,2.00,
1,-116.906566,39.135700,fallon,us,75.992,20,0,1.06,Kingston Campground
2,-159.176738,-1.224683,faanui,pf,77.756,78,0,7.63,
3,-23.271040,-24.747381,caravelas,br,78.674,70,0,3.58,
4,23.613633,18.196267,kutum,sd,79.484,40,0,5.23,
5,63.112246,30.444733,rudbar,af,70.160,22,0,3.34,
6,44.576205,2.318491,wanlaweyn,so,77.666,77,0,7.51,Haji Qasim Hotel
7,86.047779,35.753045,korla,cn,73.994,63,0,0.89,
8,114.282863,-5.740023,sumenep,id,78.728,68,0,3.58,TPU masalima


In [46]:
# NOTE: Do not change any of the code in this cell
### COMMENT IN RESPONSE: I kept the code the same but changed the Dataframe and Variable names to appropriately
### match the index and columns of the dataframe.

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotelspots_df.iterrows()]
locations = hotelspots_df[["lat", "long"]]

In [47]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…